# TP2 : tokenisation

Test de l'entrainement de modèles de tokenisation avec le logiciel sentencepiece, depuis des corpus issus du JORF.

In [2]:
import sentencepiece as spm
import pandas as pd 

## 1. Chargement des données

Données issues du JO (JORF : Journal Officile de la République Française) : 
- lois
- lois organiques
- ordonnances
- décrets
- arrếtés

On écrit l'ensemble du contenu extrait (la colonne Contenu) dans un fichier texte.
On fait de même en séparant les contenus étant directement du Droit (entourés par des guillemets) de ceux étant auxilliaires au Droit.

In [3]:
df = pd.read_csv("./input_data/jorf_2023.csv",sep='|',names=["ID texte","ID article","Nature","N° article","N° alinéa","Contenu"])
df.head()

,ID texte,ID article,Nature,N° article,N° alinéa,Contenu
0,JORFTEXT000048734585,NaN,0,NaN,0,fr/lr/loi/2023-1380/2023-12-31
1,JORFTEXT000048734585,JORFVERS000048734585,0,NaN,0,LOI n° 2023-1380 du 30 décembre 2023 visant à ...
2,JORFTEXT000048734585,JORFARTI000048734586,1,1,1,I. — Après l'article L. 2122-19 du code généra...
3,JORFTEXT000048734585,JORFARTI000048734586,1,1,2,« Art. L. 2122-19-1. — Pour assurer les foncti...
4,JORFTEXT000048734585,JORFARTI000048734586,1,1,3,II. — L'article L. 2122-19-1 du code général d...


In [4]:
# Ecrire l'ensemble du contenu dans un fichier texte
with open("input_data/jorf_2023.txt","w") as f:
    f.writelines("\n".join(df["Contenu"].to_list()))
    
# On extrait que le Droit : contenus au sein de guillemets
with open("./input_data/jorf_2023_droit.txt","w") as f:
    df2 = df[df["Contenu"].str[0] == "«"]
    data = "\n".join(df2["Contenu"].to_list())
    print("Nombre de caractères : ", len(data))
    f.writelines(data)
    
# On extrait que ce qui entoure le Droit
with open("./input_data/jorf_2023_non_droit.txt","w") as f:
    df2 = df[~(df["Contenu"].str[0] == "«")]
    data = "\n".join(df2["Contenu"].to_list())
    print("Nombre de caractères : ", len(data))
    f.writelines(data)

Nombre de caractères :  8846675
Nombre de caractères :  52633868


## 2. Entraînement des modèles

### 2.A. Test sur le nombre de tokens autorisés

In [33]:
for n in (100,1000,10000):
    spm.SentencePieceTrainer.train(input='./input_data/jorf_2023.txt', model_prefix='./models/vs{n}', vocab_size=n)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./jorf_2023.txt
  input_format: 
  model_prefix: vs100
  model_type: UNIGRAM
  vocab_size: 100
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_level: 0

In [35]:
# On encode une phrase selon les différents modèles : 100 tokens appara^t clairement insuffisant
for n in [100,1000,10000]:
    sp = spm.SentencePieceProcessor(model_file=f'./models/vs{n}.model')
    print(f"Encoding with {n} tokens :")
    print(sp.encode("Un décret en Conseil d'Etat précise les modalités d'application du présent article",out_type=str))

Encoding with 100 tokens :
['▁', 'U', 'n', '▁d', 'é', 'c', 're', 't', '▁', 'e', 'n', '▁', 'C', 'o', 'n', 's', 'e', 'i', 'l', '▁d', "'", 'E', 't', 'a', 't', '▁', 'p', 'r', 'é', 'c', 'i', 's', 'e', '▁', 'l', 'es', '▁', 'm', 'o', 'd', 'a', 'l', 'i', 't', 'é', 's', '▁d', "'", 'a', 'p', 'p', 'l', 'i', 'c', 'a', 't', 'i', 'o', 'n', '▁d', 'u', '▁', 'p', 'r', 'é', 's', 'e', 'n', 't', '▁', 'a', 'r', 't', 'i', 'c', 'l', 'e']
Encoding with 1000 tokens :
['▁Un', '▁décret', '▁en', '▁Conseil', '▁d', "'", 'Etat', '▁précise', '▁les', '▁modalités', '▁d', "'", 'application', '▁du', '▁présent', '▁', 'article']
Encoding with 10000 tokens :
['▁Un', '▁décret', '▁en', '▁Conseil', '▁d', "'", 'Etat', '▁précise', '▁les', '▁modalités', '▁d', "'", 'application', '▁du', '▁présent', '▁article']


### 2.B. Test de l'effet de n'inclure que du Droit / que des auxilliaires au Droit

In [43]:
spm.SentencePieceTrainer.train(input='./input_data/jorf_2023_droit.txt', model_prefix='./models/subset_droit', vocab_size=1000)
spm.SentencePieceTrainer.train(input='./input_data/jorf_2023_non_droit.txt', model_prefix='./models/subset_non_droit', vocab_size=1000)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./input_data/jorf_2023_droit.txt
  input_format: 
  model_prefix: ./models/subset_droit
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  d

In [44]:
sp1 = spm.SentencePieceProcessor(model_file=f'./models/subset_droit.model')
sp2 = spm.SentencePieceProcessor(model_file=f'./models/subset_non_droit.model')

In [46]:
# Comparaison de la tokenisation d'une phrase de droit et d'une non de Droit 
s1 = "« Les modalités de calcul du montant de la dotation complémentaire prise en application des présents II et III, sont définies à l'annexe 4. »"
s2 = "Arrêté du 22 décembre 2022 portant nomination au conseil d'administration de l'établissement public d'aménagement de Bordeaux-Euratlantique"

for s in [s1,s2]:
    print("#####")
    print("Tokenisation de la phrase : " + s)
    for sp in [sp1,sp2]:
        print(sp.encode(s,out_type=str))

#####
Tokenisation de la phrase : « Les modalités de calcul du montant de la dotation complémentaire prise en application des présents II et III, sont définies à l'annexe 4. »
['▁«', '▁Les', '▁modalités', '▁de', '▁calcul', '▁du', '▁montant', '▁de', '▁la', '▁d', 'ot', 'ation', '▁complémentaire', '▁prise', '▁en', '▁application', '▁des', '▁présent', 's', '▁II', '▁et', '▁III', ',', '▁sont', '▁défini', 'es', '▁à', '▁l', "'", 'annexe', '▁4', '.', '▁»']
['▁«', '▁Les', '▁modalités', '▁de', '▁calcul', '▁du', '▁montant', '▁de', '▁la', '▁d', 'ot', 'ation', '▁complémentaire', '▁p', 'ri', 's', 'e', '▁en', '▁', 'application', '▁des', '▁présent', 's', '▁II', '▁et', '▁III', ',', '▁sont', '▁défini', 'es', '▁à', '▁l', "'", 'annexe', '▁4', '.', '▁»']
#####
Tokenisation de la phrase : Arrêté du 22 décembre 2022 portant nomination au conseil d'administration de l'établissement public d'aménagement de Bordeaux-Euratlantique
['▁A', 'r', 'r', 'ê', 'té', '▁du', '▁22', '▁décembre', '▁2022', '▁portant', '▁n', 'o

On observe des différences notables. Par exemple le mot "Arrêté" n'est pas un token du modèle entraîné uniquement sur du Droit. 